# Prototype Bemchmark
For each record to be scored, the "county" specific model is read from external storage.  No model caching in memory.

* `model_load_time` (msec) is the time required to read a model file from external storage and deserialize the model object.
* `model_score_time` (msec) is the time to score one record.

## Metrics reported in this notebook are from synthetic data and **have not** been calibrated to representative dataset or model sizes.

## Notebook run-time enviornment
* **Hardware:** MacBook Pro Intel(2019), 16GB RAM, 1TB SSD drive
* **OS:** MacOS 11.6.1
* **Docker:** Docker for Desktop 4.2.0 (Mac)
* **Docker Image:** Base image: `jupyter/datascience-notebook:lab-3.2.5` with ONNX packages added

In [1]:
import glob
import os
import shutil
import pandas as pd
import numpy as np
import onnxruntime as rt
import pickle

## Setup for tests

In [2]:
# required to allow for import of project speccific utility functions
os.chdir('..')

In [3]:
# import project specific utiity functions
from utils.utils import BenchmarkDriver, load_config, actualsize_mb

In [4]:
# get configuration parameters
config = load_config('./config.yaml')
config

{'data_dir': '/Users/jim/Desktop/onnx_sandbox/data/testbed',
 'models_dir': '/Users/jim/Desktop/onnx_sandbox/models/testbed',
 'number_records': 100000,
 'number_features': 20,
 'number_informative': 14,
 'number_trees': 100,
 'fraction_for_test': 0.2,
 'number_counties': 20,
 'random_seed': 123}

In [5]:
TEST_DATA = os.path.join(config['data_dir'], 'test.parquet')
PERFORMANCE_DATA_DIR = os.path.join(config['data_dir'],'performance')
MODELS_DIR = config['models_dir']


In [6]:
# setup directory to collect performance data
shutil.rmtree(PERFORMANCE_DATA_DIR, ignore_errors=True)
os.makedirs(PERFORMANCE_DATA_DIR, exist_ok=True)

## Test Design

### Model Training
In an offline process, trained 20 Random Forest (RF) models using synthetic data generated by `sklearn.datasets.make_regression()` method.  These data sets have 20 explanatory variables (`X_00` to `X_19`) with a target variable called 'y'.  With the exception of setting `random_state` parameter to a known value, all other hyper-parameters were allowed to take their default value.

In [7]:
pd.read_parquet(os.path.join(TEST_DATA)).head()

,county,X_00,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,...,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,y
0,cnty0017,0.520762,-0.658164,0.234148,-1.949334,-1.042321,0.206521,1.468614,0.402485,-0.329271,...,1.148844,0.637367,-1.228588,-0.052683,-1.478516,-2.905790,-0.816058,-0.353458,-1.478990,-656.045166
1,cnty0002,-1.719247,0.145194,0.682192,1.195691,-0.298329,0.274303,1.889344,-1.349296,-0.027038,...,1.903116,0.869600,0.350681,0.064682,-0.017302,0.296412,-0.155503,-0.430120,0.386999,174.201584
2,cnty0005,-0.572314,-0.771446,-0.112503,0.274605,0.546655,0.919573,-1.015905,0.091013,-0.798503,...,0.244685,0.077570,0.540681,0.368747,-0.538600,0.481323,0.122542,1.925718,1.102175,79.194344
3,cnty0000,0.558090,-0.571827,1.589408,0.340502,-1.313514,0.500834,-0.114611,0.248661,-1.692944,...,-0.037938,-0.481162,1.001708,-1.416378,-1.177559,-0.419277,0.349913,-1.670872,0.788682,25.678637
4,cnty0018,-0.383470,-0.381675,1.018075,0.127476,-0.138571,-0.958881,0.165615,-0.385700,0.624114,...,1.220423,-0.427295,-0.472333,-0.883766,0.192109,0.349223,0.846627,1.677837,0.582563,58.992062


For each Random Forest model, saved two model files: sklearn (.pkl file extension) and onnx (.onnx file extension).  These files are indentified by the `county` data used to generate the RF model.  Here is a sample of saved model files.

In [8]:
os.listdir(MODELS_DIR)[:10]

['cnty0000.onnx',
 'cnty0000.pkl',
 'cnty0001.onnx',
 'cnty0001.pkl',
 'cnty0002.onnx',
 'cnty0002.pkl',
 'cnty0003.onnx',
 'cnty0003.pkl',
 'cnty0004.onnx',
 'cnty0004.pkl']

### Model Scoring
For this test selected 100 random records from the test data.  For each record performed the following:
```
# Process test batch
while there are input records:
    Read one record
    Based on 'county' value, load the RF model for that 'county'  
    Record time to load and make model useable
    Score the record
    Record process memory RSS value
    Record time to score the record

# record collected metrics
Write collected run-time metrics to an external file.
```
No explicit caching of model objects were done in this test.

## Analysis of RF Tree Structure


In [9]:
# collect data on RF tree structure
tree_metrics = []
rf_models = glob.glob(os.path.join(MODELS_DIR, '*.pkl'))
for model in rf_models:
    # get file sizes
    fp_parts = os.path.splitext(model)
    metrics = {'model': fp_parts[0].split('/')[-1]}
    metrics['sklearn_file_size_mb'] = os.path.getsize(model) / (1024 * 1024)
    metrics['onnx_file_size_mb'] = os.path.getsize(fp_parts[0] + '.onnx') / (1024 * 1024)
    
    # extract tree structure
    with open(model, 'rb') as f:
        rf = pickle.load(f)
    metrics['number_of_trees'] = len(rf.estimators_)
    tree_depth = [tree.tree_.max_depth for tree in rf.estimators_]
    metrics['tree_min_depth'] = np.min(tree_depth)
    metrics['tree_max_depth'] = np.max(tree_depth)
    metrics['tree_mean_depth'] = np.mean(tree_depth)
    
    del rf
        
    # collect metrics
    tree_metrics.append(metrics)

In [10]:
# overview of tree structure
tree_metrics_df = pd.DataFrame(tree_metrics)
tree_metrics_df

,model,sklearn_file_size_mb,onnx_file_size_mb,number_of_trees,tree_min_depth,tree_max_depth,tree_mean_depth
0,cnty0000,30.189501,18.352439,100,20,28,22.16
1,cnty0001,30.534838,18.562903,100,20,27,22.37
2,cnty0002,30.874315,18.769781,100,20,27,22.40
3,cnty0003,30.684862,18.654301,100,19,29,22.31
4,cnty0004,31.327074,19.045651,100,20,26,22.65
5,cnty0005,31.452074,19.121824,100,20,27,22.47
6,cnty0006,31.429369,19.107987,100,20,27,22.26
7,cnty0007,31.571093,19.194362,100,20,27,22.32
8,cnty0008,30.865892,18.764612,100,19,26,21.99
9,cnty0009,31.219652,18.980194,100,20,27,22.86


In [11]:
tree_metrics_of_interest = ['sklearn_file_size_mb', 'onnx_file_size_mb',
                           'number_of_trees', 'tree_min_depth', 'tree_mean_depth', 'tree_max_depth']
tree_metrics_df[tree_metrics_of_interest].describe().T

,count,mean,std,min,25%,50%,75%,max
sklearn_file_size_mb,20.0,30.901854,0.428075,30.189501,30.582933,30.832018,31.281115,31.571093
onnx_file_size_mb,20.0,18.786541,0.260859,18.352439,18.592215,18.743979,19.017661,19.194362
number_of_trees,20.0,100.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000
tree_min_depth,20.0,19.800000,0.410391,19.000000,20.000000,20.000000,20.000000,20.000000
tree_mean_depth,20.0,22.341500,0.229537,21.980000,22.215000,22.315000,22.417500,22.860000
tree_max_depth,20.0,26.850000,1.182103,25.000000,26.000000,27.000000,27.000000,30.000000


## Model Scoring Test

### Get data for test

In [12]:
test_df = pd.read_parquet(TEST_DATA)
test_df = test_df.sample(n=100, random_state=config['random_seed'])
test_df.reset_index(inplace=True)
test_df.shape

(100, 23)

In [13]:
test_df[['index','county', 'y']].head(10)

,index,county,y
0,12136,cnty0009,21.728182
1,16812,cnty0018,-191.266129
2,10072,cnty0019,18.938940
3,5850,cnty0015,57.200191
4,4320,cnty0008,-154.245728
5,4429,cnty0019,-46.112324
6,11744,cnty0016,-48.300835
7,12106,cnty0008,247.827332
8,4230,cnty0009,-219.337921
9,16858,cnty0003,-271.940826


In [14]:
metrics_of_interest = ['model_process_rss_mb', 'model_load_time_ms', 'model_score_time_ms', 'model_prediction_time_ms']

### sklearn Model Scoring

In [15]:
%%time
sklearn_driver = BenchmarkDriver(
    model_type='sklearn',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    sklearn_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns'))
    


CPU times: user 2.44 s, sys: 1.7 s, total: 4.14 s
Wall time: 4.11 s


In [16]:
# display collected performance metrics
sklearn_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'))
sklearn_metrics_df['model_prediction_time_ms'] = sklearn_metrics_df['model_load_time_ms'] + sklearn_metrics_df['model_score_time_ms']
sklearn_metrics_df.head(10)

,county_id,record_id,test_scenario,model_load_time_ms,model_score_time_ms,model_process_rss_mb,predicted_score,model_prediction_time_ms
0,cnty0009,12136,county-level,16.8461,29.1945,279.117188,46.496496,46.0406
1,cnty0018,16812,county-level,16.8120,25.4945,279.226562,-147.507202,42.3065
2,cnty0019,10072,county-level,10.9086,23.6955,279.265625,-96.099620,34.6041
3,cnty0015,5850,county-level,10.4876,27.4104,279.250000,94.128684,37.8980
4,cnty0008,4320,county-level,11.3541,23.6572,279.269531,-90.561886,35.0113
5,cnty0019,4429,county-level,10.8337,24.0255,279.269531,-33.759447,34.8592
6,cnty0016,11744,county-level,12.9625,22.2882,279.296875,-47.095236,35.2507
7,cnty0008,12106,county-level,11.9490,26.0978,279.343750,26.600595,38.0468
8,cnty0009,4230,county-level,12.0500,27.6521,279.316406,-83.146615,39.7021
9,cnty0003,16858,county-level,11.0548,24.1161,279.324219,-212.850858,35.1709


In [17]:
sklearn_metrics_df[metrics_of_interest].describe(percentiles=[.25, .5, .75, .9]).T

,count,mean,std,min,25%,50%,75%,90%,max
model_process_rss_mb,100.0,275.200625,3.036533,271.511719,271.554688,273.93750,278.682617,278.966016,279.34375
model_load_time_ms,100.0,11.377518,1.140458,10.162600,10.635925,11.10355,11.829175,12.293280,16.84610
model_score_time_ms,100.0,24.331235,1.612175,19.939300,23.312825,24.10465,25.184950,26.450360,29.19450
model_prediction_time_ms,100.0,35.708753,2.206977,30.553800,34.289950,35.30190,36.738100,38.331050,46.04060


### onnx scoring test

In [18]:
%%time
onnx_driver = BenchmarkDriver(
    model_type='onnx',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    onnx_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns').astype(np.float32).to_numpy())


CPU times: user 58.6 s, sys: 2.65 s, total: 1min 1s
Wall time: 55.9 s


In [19]:
# display collected performance metrics
onnx_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'))
onnx_metrics_df['model_prediction_time_ms'] = onnx_metrics_df['model_load_time_ms'] + onnx_metrics_df['model_score_time_ms']
onnx_metrics_df.head(10)

,county_id,record_id,test_scenario,model_load_time_ms,model_score_time_ms,model_process_rss_mb,predicted_score,model_prediction_time_ms
0,cnty0009,12136,county-level,667.5671,0.2468,449.800781,46.496500,667.8139
1,cnty0018,16812,county-level,563.6638,0.2320,461.066406,-147.507200,563.8958
2,cnty0019,10072,county-level,586.9944,0.1938,469.652344,-96.099630,587.1882
3,cnty0015,5850,county-level,660.9591,0.2482,465.214844,94.128670,661.2073
4,cnty0008,4320,county-level,591.6458,0.2700,470.441406,-90.561905,591.9158
5,cnty0019,4429,county-level,493.9440,0.2166,471.394531,-33.759453,494.1606
6,cnty0016,11744,county-level,558.1165,0.2086,471.636719,-47.095234,558.3251
7,cnty0008,12106,county-level,502.4144,0.2048,472.539062,26.600597,502.6192
8,cnty0009,4230,county-level,491.5187,0.2002,471.964844,-83.146620,491.7189
9,cnty0003,16858,county-level,582.0662,0.2003,472.097656,-212.850920,582.2665


In [20]:
onnx_metrics_df[metrics_of_interest].describe(percentiles=[.25, .5, .75, .9]).T

,count,mean,std,min,25%,50%,75%,90%,max
model_process_rss_mb,100.0,469.208203,4.908031,449.800781,466.695312,469.919922,473.418945,474.122656,475.5625
model_load_time_ms,100.0,515.832158,37.530253,468.438300,493.387950,502.534650,519.641725,566.403320,667.5671
model_score_time_ms,100.0,0.217624,0.024695,0.163600,0.200825,0.213950,0.232725,0.248390,0.2962
model_prediction_time_ms,100.0,516.049782,37.535691,468.614500,493.589150,502.748700,519.859800,566.630400,667.8139


### Differences in sklearn vs onnx predictions

In [21]:
differences = np.abs(sklearn_metrics_df['predicted_score'] - onnx_metrics_df['predicted_score'])
differences.describe()

count    1.000000e+02
mean     1.642596e-05
std      1.934064e-05
min      3.653603e-08
25%      3.462909e-06
50%      8.952596e-06
75%      2.279766e-05
max      1.167006e-04
Name: predicted_score, dtype: float64